# L’analyse fréquentiste par l’approche bag-of-words : intérêt et

limites

Lino Galiana  
2025-10-06

<div class="badge-container"><div class="badge-text">Pour essayer les exemples présents dans ce tutoriel :</div><a href="https://github.com/linogaliana/python-datascientist-notebooks/blob/main/notebooks/NLP/02_exoclean.ipynb" target="_blank" rel="noopener"><img src="https://img.shields.io/static/v1?logo=github&label=&message=View%20on%20GitHub&color=181717" alt="View on GitHub"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/vscode-python?autoLaunch=true&name=«02_exoclean»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-vscode.sh»&init.personalInitArgs=«NLP%2002_exoclean%20correction»" target="_blank" rel="noopener"><img src="https://custom-icon-badges.demolab.com/badge/SSP%20Cloud-Lancer_avec_VSCode-blue?logo=vsc&logoColor=white" alt="Onyxia"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/jupyter-python?autoLaunch=true&name=«02_exoclean»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-jupyter.sh»&init.personalInitArgs=«NLP%2002_exoclean%20correction»" target="_blank" rel="noopener"><img src="https://img.shields.io/badge/SSP%20Cloud-Lancer_avec_Jupyter-orange?logo=Jupyter&logoColor=orange" alt="Onyxia"></a>
<a href="https://colab.research.google.com/github/linogaliana/python-datascientist-notebooks-colab//blob/main//notebooks/NLP/02_exoclean.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a><br></div>

Pour avancer dans ce chapitre, nous avons besoin de quelques installations préalables:

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

Il est également utile de définir la fonction suivante, issue de notre chapitre précédent :

In [ ]:
def clean_text(doc):
    # Tokenize, remove stop words and punctuation, and lemmatize
    cleaned_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    # Join tokens back into a single string
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

# 1. Introduction

Nous avons vu, précédemment, l’intérêt de nettoyer les données pour dégrossir le volume d’information présent dans nos données non structurées. L’objectif de ce chapitre est d’approfondir notre compréhension de l’approche fréquentiste appliquée aux données textuelles. Nous allons évoquer la manière dont cette analyse fréquentiste permet de synthétiser l’information présente dans un corpus textuel. Nous allons également voir la manière dont on peut raffiner l’approche *bag of words* en tenant compte de l’ordre de la proximité des termes dans une phrase.

## 1.1 Données

Nous allons reprendre le jeu de données anglo-saxon du chapitre précédent, à savoir
des textes des auteurs fantastiques [Edgar Allan Poe](https://fr.wikipedia.org/wiki/Edgar_Allan_Poe) (*EAP*), [HP Lovecraft](https://fr.wikipedia.org/wiki/H._P._Lovecraft) (*HPL*) et [Mary Wollstonecraft Shelley](https://fr.wikipedia.org/wiki/Mary_Shelley) (*MWS*).

In [ ]:
import pandas as pd

url='https://github.com/GU4243-ADS/spring2018-project1-ginnyqg/raw/master/data/spooky.csv'
#1. Import des données
horror = pd.read_csv(url,encoding='latin-1')
#2. Majuscules aux noms des colonnes
horror.columns = horror.columns.str.capitalize()
#3. Retirer le prefixe id
horror['ID'] = horror['Id'].str.replace("id","")
horror = horror.set_index('Id')

``` python
```

# 2. La mesure TF-IDF (*term frequency - inverse document frequency*)

## 2.1 La matrice documents - termes

Comme nous l’avons évoqué précédemment, nous construisons une représentation synthétique de notre corpus comme un sac de mots dans lesquels on pioche plus ou moins fréquemment des mots selon leur fréquence d’apparition. C’est bien sûr une représentation simpliste de la réalité : les séquences de mots ne sont pas une suite aléatoire indépendante de mots.

Cependant, avant d’évoquer ces enjeux, il nous reste à aller au bout de l’approche sac de mots. La représentation la plus caractéristique de ce paradigme est la matrice document-terme, principalement utilisée pour comparer des corpus. Celle-ci consiste à créer une matrice où chaque document est représenté par la présence ou l’absence des termes de notre corpus. L’idée est de compter le nombre de fois où les mots (les termes, en colonne) sont présents dans chaque phrase ou libellé (le document, en ligne). Cette matrice fournit alors une représentation numérique des données textuelles.

Considérons un corpus constitué des trois phrases suivantes :

-   *“La pratique du tricot et du crochet”*
-   *“Transmettre la passion du timbre”*
-   *“Vivre de sa passion”*

La matrice document-terme associée à ce corpus est la suivante :

|  | crochet | de | du | et | la | passion | pratique | sa | timbre | transmettre | tricot | vivre |
|-------------------|:----:|:-:|:-:|:-:|:-:|:----:|:----:|:-:|:---:|:------:|:---:|:---:|
| La pratique du tricot et du crochet | 1 | 0 | 2 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
| Transmettre sa passion du timbre | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 0 |
| Vivre de sa passion | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 |

Chaque phrase du corpus est associée à un vecteur numérique. Par exemple,
la phrase *“La pratique du tricot et du crochet”*, qui n’a pas de sens en soi pour une machine, devient un vecteur numérique intelligible pour elle égal à `[1, 0, 2, 1, 1, 0, 1, 0, 0, 0, 1, 0]`. Ce vecteur numérique est une représentation creuse (*sparse*) du langage puisque chaque document (ligne) ne comportera qu’une petite partie du vocabulaire total (l’ensemble des colonnes). Pour tous les mots qui n’apparaîtront pas dans le document, on aura des 0, d’où un vecteur *sparse*. Comme nous le verrons par la suite, cette représentation numérique diffère grandement des approches modernes d’*embeddings*, basées sur l’idée de représentation denses.

## 2.2 Utilisation pour l’extraction d’informations

Différents documents peuvent alors être rapprochés sur la base de ces mesures. C’est l’une des manières de procéder des moteurs de recherche même si les meilleurs utilisent des approches bien plus sophistiquées. La métrique [tf-idf](https://fr.wikipedia.org/wiki/TF-IDF) (*term frequency–inverse document frequency*)
permet de calculer un score de proximité entre un terme de recherche et un
document à partir de deux composantes :

$$
\text{tf-idf}(t, d, D) = \text{tf}(t, d) \times \text{idf}(t, D)
$$

avec $t$ un terme particulier (par exemple un mot), $d$ un document particulier et $D$ l’ensemble de documents dans le corpus.

-   La partie `tf` calcule une fonction croissante de la fréquence du terme de recherche dans le document à l’étude ;

-   La partie `idf` calcule une fonction inversement proportionnelle à la fréquence du terme dans l’ensemble des documents (ou corpus).

-   La première partie (*term-frequency*, TF) est la fréquence d’apparition du terme terme $t$ dans le document $d$. Il existe des mesures de normalisation pour éviter de biaiser la mesure en cas de documents longs.

$$
\text{tf}(t, d) = \frac{f_{t,d}}{\sum_{t' \in d} f_{t',d}}
$$

où $f_{t,d}$ est le nombre brut de fois que le terme $t$ apparaît dans le document $d$ et le dénominateur est le nombre de termes dans le document $d$.

-   La seconde partie (*inverse document frequency*, IDF) mesure la rareté, ou au contraire l’aspect commun, d’un terme dans l’ensemble du corpus. Si $N$ est le nombre total de documents dans le corpus $D$, cette partie de la mesure sera

$$
\text{idf}(t, D) = \log \left( \frac{N}{|\{d \in D : t \in d\}|} \right)
$$

Le dénominateur $( |\{d \in D : t \in d\}| )$ correspond au nombre de documents contenant le terme $t$, s’il apparaît. Plus le mot est rare, plus sa présence dans un document sera surpondérée.

De nombreux moteurs de recherche utilisent cette logique pour rechercher les documents les plus pertinents pour répondre à des termes de recherche. C’est notamment le cas d’ [`ElasticSearch`](https://www.elastic.co/fr/elasticsearch), le logiciel qui permet d’implémenter des moteurs de recherche efficaces. Pour classer les documents les plus pertinents par rapport à des termes de recherche, ce logiciel utilise notamment la mesure de distance [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) qui est une version sophistiquée de la mesure TF-IDF.

## 2.3 Exemple

Prenons une illustration à partir d’un petit corpus. Le code suivant implémente une mesure TF-IDF. Celle-ci est légèrement différente de celle définie ci-dessus pour s’assurer de ne pas effectuer de division par zéro.

In [ ]:
import numpy as np

# Documents d'exemple
documents = [
    "Le corbeau et le renard",
    "Rusé comme un renard",
    "Le chat est orange comme un renard"
]

# Tokenisation
def preprocess(doc):
    return doc.lower().split()

tokenized_docs = [preprocess(doc) for doc in documents]

# Term frequency (TF)
def term_frequency(term, tokenized_doc):
    term_count = tokenized_doc.count(term)
    return term_count / len(tokenized_doc)

# Inverse document frequency (DF)
def document_frequency(term, tokenized_docs):
    return sum(1 for doc in tokenized_docs if term in doc)

# Calculate inverse document frequency (IDF)
def inverse_document_frequency(word, corpus):
    # Normalisation avec + 1 pour éviter la division par zéro
    count_of_documents = len(corpus) + 1
    count_of_documents_with_word = sum([1 for doc in corpus if word in doc]) + 1
    idf = np.log10(count_of_documents/count_of_documents_with_word) + 1
    return idf

# Calculate TF-IDF scores in each document
def tf_idf_term(term):
  tf_idf_scores = pd.DataFrame(
    [
      [
      term_frequency(term, doc),
      inverse_document_frequency(term, tokenized_docs)
      ] for doc in tokenized_docs
    ],
    columns = ["TF", "IDF"]
  )
  tf_idf_scores["TF-IDF"] = tf_idf_scores["TF"] * tf_idf_scores["IDF"]
  return tf_idf_scores

Commençons par calculer TF-IDF du mot “chat” pour chaque document. De manière
naturelle, c’est le troisième document, le seul où apparaît le mot qui a la valeur maximale :

In [ ]:
tf_idf_term("chat")

Qu’en est-il du terme renard qui apparaît dans tous les documents (dont la partie $\text{idf}$ est donc égale à 1) ? Dans ce cas, c’est le document où le mot est le plus fréquent, en l’occurrence le 2e, qui a la mesure maximale.

In [ ]:
tf_idf_term("renard")

## 2.4 Application

L’exemple précédent ne passait pas très bien à l’échelle. Heureusement, `Scikit` propose une implémentation de la recherche par vecteur TF-IDF que nous pouvons explorer avec un nouvel exercice.

> **Exercice 1 : TF-IDF : calcul de fréquence**
>
> 1.  Utiliser le vectoriseur TF-IdF de `scikit-learn` pour transformer notre corpus en une matrice `document x terms`. Au passage, utiliser l’option `stop_words` pour ne pas provoquer une inflation de la taille de la matrice. Nommer le modèle `tfidf` et le jeu entraîné `tfs`.
> 2.  Après avoir construit la matrice de documents x terms avec le code suivant, rechercher les lignes où les termes ayant la structure `abandon` sont non-nuls.
> 3.  Trouver les 50 extraits où le score TF-IDF du mot *“fear”* est le plus élevé et l’auteur associé. Déterminer la répartition des auteurs dans ces 50 documents.
> 4.  Observer les 10 scores où TF-IDF de *“fear”* sont les plus élevés
>
> <details>
>
> <summary>
>
> Aide pour la question 2
>
> </summary>
>
> ``` python
> feature_names = tfidf.get_feature_names_out()
> corpus_index = [n for n in list(tfidf.vocabulary_.keys())]
> horror_dense = pd.DataFrame(tfs.todense(), columns=feature_names)
> ```
>
> </details>

Le vectoriseur obtenu à l’issue de la question 1 est
le suivant :

Les lignes où le terme *“abandon”* apparait
sont les suivantes (question 2) :

La matrice document-terme associée à celles-ci est la suivante :

On remarque ici l’inconvénient de ne pas avoir fait de racinisation. Les variations
de *“abandon”* sont éclatées sur de nombreuses colonnes. *“abandoned”* est aussi différent d’*“abandon”* que de *“fear”*. C’est l’un des problèmes de l’approche *bag of words*.

Les 10 scores les plus élevés sont les suivants :

On remarque que les scores les plus élevés sont soient des extraits courts où le mot apparait une seule fois, soit des extraits plus longs où le mot *“fear”* apparaît plusieurs fois.

# 3. Un premier enrichissement de l’approche sac de mots : les *n-grams*

Nous avons évoqué deux principales limites à l’approche sac de mot : l’absence de prise en compte du contexte et la représentation *sparse* du langage qui rend les rapprochements entre texte parfois moyennement pertinents. Dans le paradigme du sac de mots, il est néanmoins possible de prendre en compte la séquence d’enchainement de sèmes (*tokens*) par le biais des *ngrams*.

Pour rappel, jusqu’à présent, dans l’approche *bag of words*, l’ordre des mots n’avait pas d’importance.
On considère qu’un texte est une collection de
mots tirés indépendamment, de manière plus ou moins fréquente en fonction de leur probabilité
d’occurrence. Cependant, tirer un mot particulier n’affecte pas les chances de tirer certains mots
ensuite, de manière conditionnelle.

Une manière d’introduire des liens entre les séries de *tokens* sont les *n-grams*.
On s’intéresse non seulement aux mots et à leur fréquence, mais aussi aux mots qui suivent. Cette approche est essentielle pour désambiguiser les homonymes. Le calcul de *n-grams* [1] constitue la méthode la plus simple pour tenir compte du contexte.

Pour être en mesure de mener cette analyse, il est nécessaire de télécharger un corpus supplémentaire :

[1] We use the term *bigrams* for two-word co-occurrences, *trigrams* for three-word ones, etc.

In [ ]:
import nltk
nltk.download('genesis')
nltk.corpus.genesis.words('english-web.txt')

[nltk_data] Downloading package genesis to /home/runner/nltk_data...
[nltk_data]   Package genesis is already up-to-date!

['In', 'the', 'beginning', 'God', 'created', 'the', ...]

`NLTK` offre des méthodes pour tenir compte du contexte. Pour ce faire, nous calculons les n-grams, c’est-à-dire l’ensemble des co-occurrences successives de mots n-à-n. En général, on se contente de bi-grams, au mieux de tri-grams :

-   les modèles de classification, analyse du sentiment, comparaison de documents, etc. qui comparent des n-grams avec n trop grands sont rapidement confrontés au problème de données sparse, cela réduit la capacité prédictive des modèles ;
-   les performances décroissent très rapidement en fonction de n, et les coûts de stockage des données augmentent rapidement (environ n fois plus élevé que la base de données initiale).

On va, rapidement, regarder dans quel contexte apparaît le mot `fear` dans
l’oeuvre d’Edgar Allan Poe (EAP). Pour cela, on transforme d’abord
le corpus EAP en tokens `NLTK` :

In [ ]:
eap_clean = horror.loc[horror["Author"] == "EAP"]
eap_clean = ' '.join(eap_clean['Text'])
tokens = eap_clean.split()
print(tokens[:10])
text = nltk.Text(tokens)
print(text)

['This', 'process,', 'however,', 'afforded', 'me', 'no', 'means', 'of', 'ascertaining', 'the']
<Text: This process, however, afforded me no means of...>

Vous aurez besoin des fonctions `BigramCollocationFinder.from_words` et `BigramAssocMeasures.likelihood_ratio` :

> **Exercice 2 : n-grams et contexte du mot fear**
>
> 1.  Utiliser la méthode `concordance` pour afficher le contexte dans lequel apparaît le terme `fear`.
> 2.  Sélectionner et afficher les meilleures collocations, par exemple selon le critère du ratio de vraisemblance.
>
> Lorsque deux mots sont fortement associés, cela est parfois dû au fait qu’ils apparaissent rarement. Il est donc parfois nécessaire d’appliquer des filtres, par exemple ignorer les bigrammes qui apparaissent moins de 5 fois dans le corpus.
>
> 1.  Refaire la question précédente en utilisant toujours un modèle `BigramCollocationFinder` suivi de la méthode `apply_freq_filter` pour ne conserver que les bigrammes présents au moins 5 fois. Puis, au lieu d’utiliser la méthode de maximum de vraisemblance, testez la méthode `nltk.collocations.BigramAssocMeasures().jaccard`.
>
> 2.  Ne s’intéresser qu’aux *collocations* qui concernent le mot *fear*

Avec la méthode `concordance` (question 1),
la liste devrait ressembler à celle-ci :

Même si on peut facilement voir le mot avant et après, cette liste est assez difficile à interpréter car elle recoupe beaucoup d’informations.

La `collocation` consiste à trouver les bi-grammes qui
apparaissent le plus fréquemment ensemble. Parmi toutes les paires de deux mots observées,
il s’agit de sélectionner, à partir d’un modèle statistique, les “meilleures”.
On obtient donc avec cette méthode (question 2):

Si on modélise les meilleures collocations :

Cette liste a un peu plus de sens,
on a des noms de personnages, de lieux mais aussi des termes fréquemment employés ensemble
(*Chess Player* par exemple).

En ce qui concerne les *collocations* du mot fear :

Si on mène la même analyse pour le terme *love*, on remarque que de manière logique, on retrouve bien des sujets généralement accolés au verbe :

# 4. Quelques applications

Nous venons d’évoquer un premier cas d’application de l’approche *bag of words* qui est le rapprochement de textes par leurs termes communs. Ce n’est pas le seul cas d’application de l’approche précédente. Nous allons en évoquer deux qui nous amènent vers la modélisation du langage : la reconnaissance d’entités nommées et la classification.

## 4.1 Reconnaissance des entités nommées

La [reconnaissance d’entités nommées](https://fr.wikipedia.org/wiki/Reconnaissance_d%27entit%C3%A9s_nomm%C3%A9es), également connue sous l’acronyme NER pour *named entity recognition*, est une méthode d’extraction d’information permettant d’identifier, dans un texte, la nature de certains termes dans une certain classification : lieu, personne, quantité, etc.

Pour illustrer cela,
reprenons le *Comte de Monte Cristo* et regardons sur un petit morceau de cette oeuvre ce qu’implique la reconnaissance d’entités nommées :

In [ ]:
import requests
import re

url = "https://www.gutenberg.org/files/17989/17989-0.txt"
response = requests.get(url)
response.encoding = 'utf-8'  # Assure le bon décodage
raw = response.text

dumas = (
  raw
  .split("*** START OF THE PROJECT GUTENBERG EBOOK 17989 ***")[1]
  .split("*** END OF THE PROJECT GUTENBERG EBOOK 17989 ***")[0]
)


def clean_text(text):
    text = text.lower() # mettre les mots en minuscule
    text = " ".join(text.split())
    return text

dumas = clean_text(dumas)

dumas[10000:10500]

" mes yeux. --vous avez donc vu l'empereur aussi? --il est entré chez le maréchal pendant que j'y étais. --et vous lui avez parlé? --c'est-à-dire que c'est lui qui m'a parlé, monsieur, dit dantès en souriant. --et que vous a-t-il dit? --il m'a fait des questions sur le bâtiment, sur l'époque de son départ pour marseille, sur la route qu'il avait suivie et sur la cargaison qu'il portait. je crois que s'il eût été vide, et que j'en eusse été le maître, son intention eût été de l'acheter; mais je lu"

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("fr_core_news_sm")
doc = nlp(dumas[15000:17000])
# displacy.render(doc, style="ent", jupyter=True)

La reconnaissance d’entités nommées disponible
par défaut dans les librairies généralistes est souvent décevante ; il est
souvent nécessaire d’enrichir les règles par défaut
par des règles *ad hoc*, propres à chaque corpus.

En pratique, récemment,
l’approche de reconnaissance d’entités nommées a été utilisée
par Etalab pour [pseudonymiser des documents administratifs](https://guides.etalab.gouv.fr/pseudonymisation/#sommaire). Il s’agit d’identifier certaines informations sensibles (état civil, adresse…) par reconnaissance d’entités pour les remplacer par des pseudonymes.

## 4.2 Classification de données textuelles : l’algorithme `Fasttext`

`Fasttext` est un réseau de neurone à une couche développé par Meta en 2016 pour faire de la classification de texte ou de la modélisation de langage. Comme nous allons pouvoir le voir, ce modèle va nous amener à faire la transition avec une modélisation plus raffinée du langage, bien que celle de `Fasttext` soit beaucoup plus frustre que celle des grands modèles de langage (LLM). L’un des principaux cas d’utilisation de `Fasttext` est la classification supervisée de données textuelles. Il s’agit, à partir d’un texte, de déterminer sa catégorie d’appartenance. Par exemple, à partir d’un texte de chanson, s’il s’agit de rap ou de rock. Ce modèle est supervisé puisqu’il apprend à reconnnaître les *features*, en l’occurrence des morceaux de texte, qui permettent d’avoir une bonne performance de prédiction sur le jeu d’entraînement puis de test.

Le concept de *feature* peut sembler étonnant pour des données textuelles, qui sont, par essence, non structurées. Pour des données structurées, la démarche évoquée dans la partie [modélisation](../../content/course/modelisation/index.qmd) apparaîssait naturelle : nous avions des variables observées dans nos données pour les *features* et l’algorithme de classification consistait à trouver la combinaison entre elles permettant de prédire, au mieux, le *label*. Avec des données textuelles, ce concept de *feature* observé n’est plus naturel. Il est nécessaire de le construire à partir d’un texte. L’information destructurée devient de l’information structurée. C’est là qu’interviennent les concepts que nous avons vu jusqu’à présent.

`FastText` est un *“sac de n-gram”*. Il considère donc que les *features* sont à construire à partir des mots de notre corpus mais aussi des *ngrams* à plusieurs niveaux. L’architecture générale de `FastText` ressemble à celle-ci :

<figure>
<img src="https://raw.githubusercontent.com/InseeFrLab/formation-mlops/main/slides/img/diag-fasttext.png" alt="Illustration de l’architecture de FastText" />
<figcaption aria-hidden="true">Illustration de l’architecture de <code>FastText</code></figcaption>
</figure>

Ce qui nous intéresse ici est la partie gauche de ce diagramme, la *“feature extraction”* car la partie *embedding* correspond à des concepts que nous verrons lors des prochains chapitres. Avec l’exemple de cette figure, on voit que le texte *“Business engineering and services”* est tokenisé comme nous avons pu le voir plus tôt en mots. Mais `Fasttext` créé également des ngrams à plusieurs niveaux. Par exemple, il va créer des bigrams de mots : *“Business engineering”*, *“engineering and”*, *“and services”*. mais aussi des quadrigrammes de caractères *“busi”*, *“usin”* et *“sine”*. Ensuite, `Fasttext` transformera tous ces termes en vecteurs numériques. Contrairement à ce que nous avons vu jusqu’à présent, ces vecteurs ne sont pas des fréquences d’apparition dans le corpus (principe de la matrice origine-document), ce sont des plongements de mots (*word embedding*). Nous découvrirons leurs principes dans les prochains chapitres.

Ce modèle `Fasttext` est très utilisé dans la statistique publique car de nombreuses sources de données textuelles sont à hiérarchiser dans des nomenclatures agrégées.

Pour voir une démonstration interactive d’un tel modèle, se rendre sur la [page du site](../../content/NLP/02_exoclean.qmd) associée à ce notebook.